In [1]:
#include packages
import pandas as pd
import geopandas as gpd #geopandas for dataframes with spatial info
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt #for creating plots

import os 
import seaborn as sns #for styling the plots
import statsmodels.formula.api as smf
import warnings
import urllib.request
warnings.filterwarnings('ignore')
%matplotlib inline 

In [2]:
censusData = pd.read_csv("../nyc_data_clean.csv")
censusData.head()

,zipcode,disadvantaged,index_change,t1_index_perc,t1_age_perc,t1_edu_perc,t1_income_perc,t1_house_perc,t2_index_perc,t2_age_perc,...,dtv_16,dtv_17,dtv_18,dtv_19,dtv_20,dtv_21,dtv_22,dtv_23,dtv_24,dtv_25
0,11211.0,True,44.811321,33.962264,72.641509,41.037736,23.584906,26.415094,78.773585,91.509434,...,0.212351,-0.151956,0.035294,-0.156758,-0.032275,-0.065525,0.020810,0.082637,0.061530,0.228933
1,11216.0,True,46.226415,25.000000,50.000000,26.886792,25.943396,25.000000,71.226415,87.264151,...,0.217238,-0.133902,0.030539,-0.165824,-0.028159,-0.065424,0.025795,0.064103,0.062932,0.221863
2,11237.0,True,38.443396,27.122642,71.226415,15.094340,21.698113,32.547170,65.566038,86.792453,...,0.207388,-0.133154,0.037539,-0.156789,-0.029169,-0.060019,0.027909,0.077233,0.052866,0.223541
3,10027.0,False,8.726415,52.830189,78.301887,77.358491,29.716981,25.471698,61.556604,75.000000,...,0.215143,-0.132584,0.031544,-0.172003,-0.040413,-0.063051,0.019255,0.059278,0.068294,0.213055
4,11105.0,False,5.896226,75.707547,79.245283,67.452830,45.283019,67.924528,81.603774,91.037736,...,0.216316,-0.136701,0.025876,-0.176122,-0.035760,-0.049296,0.025229,0.056881,0.050579,0.213737


In [3]:
censusData.columns

Index(['zipcode', 'disadvantaged', 'index_change', 't1_index_perc',
       't1_age_perc', 't1_edu_perc', 't1_income_perc', 't1_house_perc',
       't2_index_perc', 't2_age_perc', 't2_edu_perc', 't2_income_perc',
       't2_house_perc', 'count_listings', 'count_reviews', 'price', 'bedrooms',
       'star_rating', 'review_rating_location', 'review_len',
       'location_words_perc', 'sent_comp', 'sent_comp_location', 'lda_1',
       'lda_2', 'lda_3', 'lda_4', 'lda_5', 'dtv_1', 'dtv_2', 'dtv_3', 'dtv_4',
       'dtv_5', 'dtv_6', 'dtv_7', 'dtv_8', 'dtv_9', 'dtv_10', 'dtv_11',
       'dtv_12', 'dtv_13', 'dtv_14', 'dtv_15', 'dtv_16', 'dtv_17', 'dtv_18',
       'dtv_19', 'dtv_20', 'dtv_21', 'dtv_22', 'dtv_23', 'dtv_24', 'dtv_25'],
      dtype='object')

In [4]:
censusData.sort_values(by="zipcode")

,zipcode,disadvantaged,index_change,t1_index_perc,t1_age_perc,t1_edu_perc,t1_income_perc,t1_house_perc,t2_index_perc,t2_age_perc,...,dtv_16,dtv_17,dtv_18,dtv_19,dtv_20,dtv_21,dtv_22,dtv_23,dtv_24,dtv_25
53,10001.0,False,0.000000,85.377358,84.905660,86.792453,67.924528,82.547170,85.377358,83.490566,...,0.195505,-0.159639,0.058055,-0.144217,-0.047372,-0.071903,0.006895,0.062349,0.075384,0.213693
5,10002.0,True,6.367925,27.122642,58.962264,40.094340,23.113208,18.396226,33.490566,70.283019,...,0.193486,-0.153979,0.041070,-0.157923,-0.033471,-0.054882,0.005096,0.090989,0.073124,0.219789
28,10003.0,False,0.943396,91.509434,94.339623,92.452830,87.735849,89.622642,92.452830,88.207547,...,0.202329,-0.152814,0.041396,-0.156848,-0.038542,-0.070566,0.001861,0.077743,0.073805,0.219519
50,10004.0,False,0.000000,100.000000,99.056604,96.698113,99.528302,99.056604,100.000000,99.056604,...,0.190938,-0.148677,0.052952,-0.137566,-0.045577,-0.092392,0.015691,0.063466,0.079133,0.218218
54,10005.0,False,-0.471698,99.528302,100.000000,98.113208,96.226415,98.584906,99.056604,98.113208,...,0.196403,-0.155750,0.040017,-0.138471,-0.048002,-0.085510,0.018038,0.051816,0.077867,0.220492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,11436.0,True,23.584906,31.132075,18.867925,30.188679,49.528302,54.716981,54.716981,52.830189,...,0.240794,-0.129851,0.022824,-0.149151,-0.030478,-0.057380,0.031142,0.044707,0.021392,0.162398
91,11691.0,True,1.179245,20.047170,17.924528,38.207547,25.000000,28.301887,21.226415,45.754717,...,0.234217,-0.124982,0.024806,-0.152665,-0.034420,-0.077784,0.046859,0.052027,0.031679,0.166582
116,11692.0,True,-12.735849,32.075472,40.566038,39.150943,42.452830,36.320755,19.339623,18.867925,...,0.238092,-0.126698,0.022934,-0.136239,-0.028825,-0.078660,0.056190,0.047880,0.048315,0.180066
132,11693.0,True,4.009434,19.339623,20.754717,29.245283,34.433962,20.754717,23.349057,19.339623,...,0.242257,-0.121459,-0.006735,-0.116525,-0.014730,-0.064085,0.045757,0.049323,0.063468,0.160977


In [6]:
censusData[['zipcode', 'disadvantaged', 'index_change', 't2_index_perc', 't2_age_perc', 't2_edu_perc', 't2_income_perc','t2_house_perc']].head()

,zipcode,disadvantaged,index_change,t2_index_perc,t2_age_perc,t2_edu_perc,t2_income_perc,t2_house_perc
0,11211.0,True,44.811321,78.773585,91.509434,66.037736,50.943396,71.226415
1,11216.0,True,46.226415,71.226415,87.264151,71.226415,42.924528,39.622642
2,11237.0,True,38.443396,65.566038,86.792453,43.867925,37.264151,58.490566
3,10027.0,False,8.726415,61.556604,75.000000,79.245283,34.433962,30.188679
4,11105.0,False,5.896226,81.603774,91.037736,78.301887,58.490566,76.415094


In [10]:
sorted(censusData["t1_income_perc"].values)

[6.132075471698112,
 6.132075471698112,
 6.132075471698112,
 6.132075471698112,
 6.132075471698112,
 12.264150943396224,
 12.735849056603776,
 13.20754716981132,
 14.150943396226412,
 14.62264150943396,
 15.094339622641508,
 15.566037735849054,
 16.037735849056602,
 16.50943396226415,
 16.9811320754717,
 17.452830188679247,
 17.924528301886788,
 18.39622641509434,
 18.867924528301888,
 19.339622641509436,
 19.81132075471698,
 20.28301886792453,
 20.75471698113208,
 21.22641509433962,
 21.698113207547173,
 22.16981132075472,
 22.641509433962263,
 23.11320754716981,
 23.58490566037736,
 24.05660377358491,
 24.52830188679245,
 25.0,
 25.47169811320755,
 25.94339622641509,
 26.41509433962264,
 26.88679245283019,
 27.358490566037737,
 27.83018867924528,
 28.301886792452827,
 28.77358490566038,
 29.24528301886793,
 29.716981132075468,
 30.188679245283016,
 30.66037735849057,
 31.13207547169811,
 31.603773584905664,
 32.075471698113205,
 32.54716981132077,
 33.0188679245283,
 33.4905660377358

In [13]:
(91.509434+66.037736+50.943396+71.226415)/4

69.92924525000001

In [14]:
censusData[['zipcode', 'disadvantaged', 'index_change']].sort_values(by="index_change")

,zipcode,disadvantaged,index_change
55,10280.0,False,-87.500000
82,11355.0,False,-38.679245
118,10307.0,False,-33.962264
71,10301.0,False,-31.603774
133,10312.0,False,-30.660377
...,...,...,...
29,11205.0,True,33.490566
21,11222.0,True,37.735849
2,11237.0,True,38.443396
0,11211.0,True,44.811321


In [16]:
zipcode_age_edu_income = censusData[['zipcode', 't2_age_perc', 't2_edu_perc', 't2_income_perc']]

In [17]:
zipcode_age_edu_income.rename(columns={"t2_age_perc": "age_perc", "t2_edu_perc": "edu_perc", "t2_income_perc": "income_perc"}, inplace=True)
zipcode_age_edu_income.head()

,zipcode,age_perc,edu_perc,income_perc
0,11211.0,91.509434,66.037736,50.943396
1,11216.0,87.264151,71.226415,42.924528
2,11237.0,86.792453,43.867925,37.264151
3,10027.0,75.000000,79.245283,34.433962
4,11105.0,91.037736,78.301887,58.490566


In [19]:
zipcode_age_edu_income.to_csv("zipcode_age_edu_income.csv", index=None)